In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#define image size and batch size
IMG_SIZE=224
BATCH_SIZE=32

In [ ]:
#Define data generators for train, validation and test sets
train_datagen = ImageDataGenerator(rescale=1./225,validation_split=0.2)
train_generator= train_datagen.flow_from_directory('/content/drive/MyDrive/Brain_Tumor_Detection/Brain_Tumor_Detection/train',
    target_size = (IMG_SIZE, IMG_SIZE),
    batch_size = BATCH_SIZE,
    class_mode = 'binary',
    subset = 'training'
)
val_generator=train_datagen.flow_from_directory('/content/drive/MyDrive/Brain_Tumor_Detection/Brain_Tumor_Detection/test',
    target_size = (IMG_SIZE, IMG_SIZE),
    batch_size = BATCH_SIZE,
    class_mode = 'binary',
    subset = 'validation'
 )
test_datagen = ImageDataGenerator(rescale=1./225)
test_generator= test_datagen.flow_from_directory('/content/drive/MyDrive/Brain_Tumor_Detection/Brain_Tumor_Detection/test',
    target_size = (IMG_SIZE, IMG_SIZE),
    batch_size = BATCH_SIZE,
    class_mode = 'binary',

)


Found 2446 images belonging to 2 classes.
Found 12 images belonging to 1 classes.
Found 60 images belonging to 1 classes.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#define the model
model=keras.Sequential([
    layers.Conv2D(32,(3,3),activation='relu',input_shape=(IMG_SIZE,IMG_SIZE,3)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(128,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(128,activation='relu'),
    layers.Dense(1,activation='sigmoid')
  ])

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
history=model.fit(train_generator,validation_data=val_generator,epochs=5)

Epoch 1/5
77/77 [==============================] - 407s 5s/step - loss: 0.5444 - accuracy: 0.7858 - val_loss: 1.0861 - val_accuracy: 0.4167
Epoch 2/5
77/77 [==============================] - 272s 4s/step - loss: 0.2316 - accuracy: 0.9113 - val_loss: 1.5477 - val_accuracy: 0.4167
Epoch 3/5
77/77 [==============================] - 274s 4s/step - loss: 0.1094 - accuracy: 0.9644 - val_loss: 1.3438 - val_accuracy: 0.5833
Epoch 4/5
77/77 [==============================] - 274s 4s/step - loss: 0.0584 - accuracy: 0.9820 - val_loss: 2.4019 - val_accuracy: 0.5833
Epoch 5/5
77/77 [==============================] - 274s 4s/step - loss: 0.0200 - accuracy: 0.9947 - val_loss: 2.3633 - val_accuracy: 0.5833


In [ ]:
model.save("Model.h5","label.txt")

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

#load the saved model
model=load_model('/content/Model.h5')

#load and preprocess the test image
test_image_path='/content/drive/MyDrive/Brain_Tumor_Detection/Brain_Tumor_Detection/test/pred/pred11.jpg'
img=image.load_img(test_image_path,target_size=(224,224))
img_array=image.img_to_array(img)
img_array=np.expand_dims(img_array ,axis=0)  #batch dimension
img_array /=255.  #normalise the pixel values
#make predictions
prediction=model.predict(img_array)
if prediction<0.5:
  print("Prediction: No Tumor (probablity:",prediction[0][0],")")
else:
  print("Prediction: Tumor present (probability:",prediction[0][0],")")

1/1 [==============================] - 0s 175ms/step
Prediction: Tumor present (probability: 0.97597086 )
